In [1]:
from PIL import Image
import numpy as np
import torch

from DeepPrint.model import DeepPrintNet

In [2]:
model = DeepPrintNet(2500)

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print('Trainable Params:', int(params))

Trainable Params: 67841713


In [3]:
sample_input = torch.randn(1, 1, 448, 448)
output = model(sample_input)

In [ ]:
embedding, map, aligment, aligned, R1, R2, logits_r1, logits_r2 = output.values()
print('embedding shape:', embedding.shape)
print('map shape:', map.shape)
print('aligment shape:', aligment.shape)
print('aligned shape:', aligned.shape)
print('R1 shape:', R1.shape)
print('R2 shape:', R2.shape)
print('logits r1 shape:', logits_r1.shape)
print('logits r1 shape:', logits_r2.shape)

embedding shape: torch.Size([1, 192])
map shape: torch.Size([1, 6, 192, 192])
aligment shape: torch.Size([1, 3])
aligned shape: torch.Size([1, 1, 448, 448])
R1 shape: torch.Size([1, 96])
R2 shape: torch.Size([1, 96])
logits r1 shape: torch.Size([1, 2500])
logits r1 shape: torch.Size([1, 2500])


In [5]:
import time

model.eval()
model.to('cpu')

# Create dummy input and add batch dimension
input_tensor = torch.rand((1, 1, 448, 448)).to('cpu')

# Warm-up runs
for _ in range(3):
    with torch.no_grad():
        _ = model(input_tensor)

# Timed runs
n_runs = 20
times = []

for _ in range(n_runs):
    start_time = time.perf_counter()
    with torch.no_grad():
        _ = model(input_tensor)
    end_time = time.perf_counter()
    times.append(end_time - start_time)

avg_time = sum(times) / len(times)
fps = 1 / avg_time

print(f"Average inference time over {n_runs} runs (CPU): {avg_time:.4f} seconds")
print(f"Approximate FPS: {fps:.2f}")

Average inference time over 20 runs (CPU): 0.4135 seconds
Approximate FPS: 2.42


In [6]:
create_model_image = False

if create_model_image:
    from torchviz import make_dot

    sample_input = torch.randn(1, 1, 448, 448)
    output = model(sample_input)
    make_dot(output['embedding'], params=dict(model.named_parameters())).render("deepprint_graph", format="png")
